# Support Vector Machines


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix  
import re
import string

from sklearn.model_selection import train_test_split
from sklearn import svm


## Prepara los datos

In [ ]:
## Funcionalidad dependiente del sistema operativo
import os
data_directory = os.path.join('../data', '')
admission_filepath = os.path.join(data_directory, 'microfinance_tweets.csv')
data = pd.read_csv(admission_filepath, encoding='ISO-8859-1')
#carga datos guardados localmente
#data = pd.read_csv("microfinance_tweets.csv", encoding="ISO-8859-1")

In [ ]:
data.loc[data['Sentiment'] == 'negative', 'Sentiment'] = -1
data.loc[data['Sentiment'] == 'neutral', 'Sentiment'] = 0
data.loc[data['Sentiment'] == 'positive', 'Sentiment'] = 1

In [ ]:
data.head()

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train['Comments'])
test_features =  vectorizer.transform(test['Comments'])

Hemos vectorizado nuestros datos de modo tal que cada índice corresponde tanto a una palabra como a la frecuencia con la que esa palabra aparece en el texto

In [ ]:
print(train_features[0])

## SVM Lineal

Hay muchos tipos de SVMs, pero primero probaremos un SVM lineal, el más básico. Esto significa que el borde de descisión será lineal. <br>

Hay otro input llamado decision_function_shape. Con dos opciones: uno vs el resto, y uno vs uno. Estas se relacionan con como el borde de desición separa puntos, si separa puntos negativos del resto, o puntos negativos de puntos neutrales, etc. (https://pythonprogramming.net/support-vector-machine-parameters-machine-learning-tutorial/). El por defecto es uno vs el resto. Uno vs el resto toma menos recursos computacionales pero podría ser afectado por datos extremos y no lo hacen bien con datos inbalanceados.


In [ ]:
clf = svm.SVC(kernel='linear')  
clf.fit(train_features, train['Sentiment'])

In [ ]:
y_train = clf.predict(train_features)  

In [ ]:
print(confusion_matrix(train['Sentiment'],y_train)) 
print(classification_report(train['Sentiment'],y_train))  

In [ ]:
y_pred = clf.predict(test_features)  

In [ ]:
print(confusion_matrix(test['Sentiment'],y_pred)) 
print(classification_report(test['Sentiment'],y_pred))  

¿Qué piensas del desempeño del SVM? Podemos también ajustar gamma para tomar en cuenta el sobre ajuste, pero no pareciera que estamos sobre ajustando demasiado dado los desempeños en los datos de test y entrenamiento.


Recuerda que los vectores de soporte son los puntos de datos que están más cercanos a la superficie de decisión (o hyperplano). Podemos determinar cuales son estos puntos para cada clase que estamos clasificando, notando que tenemos 3 clases: negativo, neutro, y positivo.


In [ ]:
print(clf.support_vectors_)

Podemos corroborar el número de puntos en cada clase usando otra función. Aquí vemos que la mayoría de los vectores de soporte están en nuestra última clase, la clase positiva.


In [ ]:
clf.n_support_

Podemos también encontrar el vector de soporte en nuestra data original usando los índices proporcionados en clf.support_


In [ ]:
clf.support_

In [ ]:
print(train_features[8])

##  SVM No lineal

Podemos probar distintos tipos de kernel, con rbf siendo el gausiano, y sigmoid siendo similar a la función sigmoide de regresión logística.

In [ ]:
clf = svm.SVC(kernel='rbf')  
clf.fit(train_features, train['Sentiment'])

y_pred = clf.predict(test_features)  

In [ ]:
print(confusion_matrix(test['Sentiment'],y_pred)) 
print(classification_report(test['Sentiment'],y_pred))  

In [ ]:
clf = svm.SVC(kernel='sigmoid')  
clf.fit(train_features, train['Sentiment'])

y_pred = clf.predict(test_features)  

In [ ]:
print(confusion_matrix(test['Sentiment'],y_pred)) 
print(classification_report(test['Sentiment'],y_pred))  

Parece que el SVM lineal se desempeña mejor en este modelo tanto de la perspectiva de presición y recall. Recuerda que presición es la exactitud de la predicción y recall is que tanto del espacio positivo real estamos capturando.


¿Qué nos dice esto sobre nuestros datos?

Fuente: https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/, https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html, https://gist.github.com/WittmannF/60680723ed8dd0cb993051a7448f7805